In [ ]:
from pydantic import ValidationError

import app.modules.database.schemas.curriculum_neo as neo_curriculum
import app.modules.database.tools.xl_planner_tools as planner
import app.modules.database.tools.neontology_tools as neon
import app.modules.database.tools.neo4j_driver_tools as driver

In [ ]:
neo4j_driver = driver.get_neo4j_driver()
neon.init_neo4j_connection()


In [ ]:
# Create SchoolNodes from a list
schools = [
    #SchoolNode(school_id='KCAR', school_name='King Charles School', school_org_type='Academy', school_address='King Charles School, Holly Hall Road, Dudley, West Midlands, DY2 0TZ', school_website='https://www.kingcharlesschool.co.uk/'),
    #SchoolNode(school_id='KEV', school_name='Kevlar Academy', school_org_type='Academy', school_address='Kevlar Academy, 123 Fake Street, London, SW1A 1AA', school_website='https://www.kevlaracademy.com/')
]
# Now create a routine to create schools from a list
for school in schools:
    neon.create_or_merge_neontology_node(school, operation='create')

# Create DepartmentNodes from a list
departments = [
    DepartmentNode(department_id='SCI', department_name='Science'),
    DepartmentNode(department_id='MAT', department_name='Mathematics'),
    DepartmentNode(department_id='ENG', department_name='English'),
    DepartmentNode(department_id='HIS', department_name='History')
]
# Now create a routine to create departments from a list
for department in departments:
    neon.create_or_merge_neontology_node(department, operation='create')

# Create KeyStageNodes from a list
key_stages = [
    KeyStageNode(key_stage_level=3, key_stage='KS3'),
    KeyStageNode(key_stage_level=4, key_stage='KS4'),
    KeyStageNode(key_stage_level=5, key_stage='KS5')
]
# Now create a routine to create key stages from a list
for key_stage in key_stages:
    neon.create_or_merge_neontology_node(key_stage, operation='create')
# Create SubjectNodes from a list
subjects = [
    neo_curriculum.SubjectNode(subject_id='SCI', subject_name='Science'),
    neo_curriculum.SubjectNode(subject_id='PHY', subject_name='Physics'),
    neo_curriculum.SubjectNode(subject_id='CHE', subject_name='Chemistry'),
    neo_curriculum.SubjectNode(subject_id='BIO', subject_name='Biology'),
    neo_curriculum.SubjectNode(subject_id='MAT', subject_name='Mathematics'),
    neo_curriculum.SubjectNode(subject_id='ENG', subject_name='English'),
    neo_curriculum.SubjectNode(subject_id='HIS', subject_name='History')
]
# Now create a routine to create subjects from a list
for subject in subjects:
    neon.create_or_merge_neontology_node(subject, operation='create')


In [ ]:
planner = planner.get_excel_sheets('planner.xlsx ')

topic_df = planner['topiclookup_df']
lesson_df = planner['lessonlookup_df']
statement_df = planner['statementlookup_df']


In [ ]:
topic_excel_node_list = []
topic_excel_helper_list = []
default_values = {
    'topic_assessment_type': 'Null'
    }

for index, row in topic_df.iterrows():
    # Filter and map the row to TopicNode fields
    topic_node_data = {
        'topic_id': row.get('TopicID'),
        'topic_title': row.get('TopicTitle'),
        'total_number_of_lessons_for_topic': row.get('TotalNumberOfLessonsForTopic'),
        'topic_type': row.get('TopicType'),
        'topic_assessment_type': row.get('TopicAssessmentType')
    }
    logging.pedantic(f"Processing row {index}: {topic_node_data}")
    topic_excel_helper_data = {
        'topic_source': row.get('TopicSource'),
        'topic_department': row.get('TopicDepartment'),
        'topic_key_stage': row.get('TopicKeyStage'),
        'topic_year': row.get('TopicYear'),
        'topic_subject': row.get('TopicSubject'),
        'topic_sequence': row.get('TopicSequence')
    }
    # Replace NaN values with defaults
    topic_excel_node_data_processed = replace_nan_with_default(topic_node_data, default_values)
    topic_excel_helper_data_processed = replace_nan_with_default(topic_excel_helper_data, default_values)
    logging.pedantic(f"Processed row {index}: {topic_excel_helper_data_processed}")

    # Create a TopicNode instance for each row
    try:
        topic_node = TopicNode(**topic_excel_node_data_processed)
        logging.pedantic(f"TopicNode instance created for row {index}: {topic_node}")
        combined_data = {
            'node': topic_node,
            'helper': topic_excel_helper_data
        }
        logging.pedantic(f"Combined data for row {index}: {combined_data}")
        topic_excel_node_list.append(combined_data)
        logging.pedantic(f"Appended new combined data to topic_excel_node_list, current length: {len(topic_excel_node_list)}")
    except ValidationError as e:
        logging.error(f"Validation error for row {index}: {e}")


# Then, use the create_or_merge_neontology_node function to add these nodes to your Neo4j database
for node_data in topic_excel_node_list:
    try:
        logging.pedantic(f"Processing node: {node_data}")
        neon.create_or_merge_neontology_node(node_data['node'], operation='merge')
        logging.pedantic(f"Node processed: {node_data}")
    except Exception as e:
        logging.error(f"Error in processing node: {e}")


In [ ]:
topic_lesson_excel_node_list = []
topic_lesson_excel_helper_list = []
default_values = {
    'topic_lesson_title': 'Null',  # Corrected default value key
    'topic_lesson_type': 'Null',  # Corrected default value key
    'topic_lesson_length': 1,             # Corrected default value key
    'topic_lesson_suggested_activities': 'Null',  # Corrected default value key
    'topic_lesson_skills_learned': 'Null',  # Corrected default value key
    'topic_lesson_weblinks': 'Null',   # Corrected default value key
}

for index, row in lesson_df.iterrows():
    # Filter and map the row to TopicLessonNode fields
    topic_lesson_node_data = {
        'topic_lesson_id': row.get('LessonID'),
        'topic_lesson_title': row.get('LessonTitle', default_values['topic_lesson_title']),
        'topic_lesson_type': row.get('TopicLessonType', default_values['topic_lesson_type']),
        'topic_lesson_length': row.get('SuggestedNumberOfPeriodsForLesson', default_values['topic_lesson_length']),
        'topic_lesson_suggested_activities': row.get('SuggestedActivities', default_values['topic_lesson_suggested_activities']),
        'topic_lesson_skills_learned': row.get('SkillsLearned', default_values['topic_lesson_skills_learned']),
        'topic_lesson_weblinks': row.get('TopicLessonWeblinks', default_values['topic_lesson_weblinks'])
        }
    logging.pedantic(f"topic_lesson_node_data: {topic_lesson_node_data}")
    topic_lesson_excel_helper_data = {
        'topic_lesson_source': row.get('TopicSource'),
        'topic_lesson_department': row.get('TopicDepartment'),
        'topic_lesson_key_stage': row.get('TopicKeyStage'),
        'topic_lesson_topic_id': row.get('TopicID'),
        'topic_lesson_topic_year': row.get('TopicYear'),
        'topic_lesson_topic_subject': row.get('TopicSubject'),
        'topic_lesson_topic_sequence': row.get('TopicSequence'),
        'topic_lesson_lesson_sequence': row.get('LessonSequence'),
        'topic_lesson_learning_objective': row.get('LessonLearningObjective'),
        }
    logging.pedantic(f"topic_lesson_excel_helper_list: {topic_lesson_excel_helper_list}")

    # Replace NaN values with defaults
    topic_lesson_excel_node_data_processed = replace_nan_with_default(topic_lesson_node_data, default_values)
    logging.pedantic(f"topic_lesson_excel_node_list_processed_data: {topic_lesson_excel_node_data_processed}")
    topic_lesson_excel_helper_data_processed = replace_nan_with_default(topic_lesson_excel_helper_data, default_values)
    logging.pedantic(f"topic_lesson_excel_helper_list_processed_data: {topic_lesson_excel_helper_data_processed}")

    # Create a TopicLessonNode instance for each row
    try:
        topic_lesson_node = TopicLessonNode(**topic_lesson_excel_node_data_processed)
        logging.pedantic(f"topic_lesson_node: {topic_lesson_node}")
        combined_data = {
            'node': topic_lesson_node,
            'helper': topic_lesson_excel_helper_data
        }
        logging.pedantic(f"combined_data: {combined_data}")
        topic_lesson_excel_node_list.append(combined_data)
        logging.pedantic(f"Appended new combined data to topic_lesson_excel_node_list, current length: {len(topic_lesson_excel_node_list)}")
    except ValidationError as e:
        logging.error(f"Validation error for row {index}: {e}")
        
for node_data in topic_lesson_excel_node_list:
    try:
        neon.create_or_merge_neontology_node(node_data['node'], operation='create')
        logging.pedantic(f"TopicLessonNode instance created for row {index}: {topic_lesson_node}")
    except Exception as e:
        logging.error(f"Error in processing node: {e}")


In [ ]:
learning_statement_excel_node_list = []
learning_statement_excel_helper_list = []
default_values = {
    # Add default values for fields that might contain NaN
    'lesson_learning_statement': 'Null',
    'lesson_learning_statement_type': 'Student learning outcome'
}

for index, row in statement_df.iterrows():
    # Filter and map the row to LearningStatementNode fields
    learning_statement_node_data = {
        'lesson_learning_statement_id': row.get('LearningOutcomeID'),
        'lesson_learning_statement': row.get('LearningOutcomeStatement', default_values['lesson_learning_statement']),
        'lesson_learning_statement_type': row.get('LearningStatementType', default_values['lesson_learning_statement_type']),
    }
    logging.pedantic(f"learning_statement_node_data: {learning_statement_node_data}")
    learning_statement_excel_helper_data = {
        'lesson_learning_statement_source': row.get('TopicSource'),
        'lesson_learning_statement_department': row.get('TopicDepartment'),
        'lesson_learning_statement_key_stage': row.get('TopicKeyStage'),
        'lesson_learning_statement_topic_id': row.get('TopicID'),
        'lesson_learning_statement_lesson_id': row.get('LessonID'),
        'lesson_learning_statement_topic_year': row.get('TopicYear'),
        'lesson_learning_statement_topic_subject': row.get('TopicSubject'),
        'lesson_learning_statement_topic_sequence': row.get('TopicSequence'),
        'lesson_learning_statement_lesson_sequence': row.get('LessonSequence'),
        'lesson_learning_statement_sequence': row.get('LearningOutcomeSequence')            
    }
    logging.pedantic(f"learning_statement_excel_helper_list: {learning_statement_excel_helper_list}")

    # Replace NaN values with defaults
    learning_statement_excel_node_data_processed = replace_nan_with_default(learning_statement_node_data, default_values)
    logging.pedantic(f"processed_data: {learning_statement_excel_node_data_processed}")
    learning_statement_excel_helper_data_processed = replace_nan_with_default(learning_statement_excel_helper_data, default_values)
    logging.pedantic(f"processed_data: {learning_statement_excel_helper_data_processed}")

    # Create a LearningStatementNode instance for each row
    try:
        learning_statement_node = LearningStatementNode(**learning_statement_excel_node_data_processed)
        logging.pedantic(f"learning_statement_node: {learning_statement_node}")
        combined_data = {
            'node': learning_statement_node,
            'helper': learning_statement_excel_helper_data
        }
        logging.pedantic(f"combined_data: {combined_data}")
        learning_statement_excel_node_list.append(combined_data)
        logging.pedantic(f"Appended new combined data to learning_statement_excel_node_list, current length: {len(learning_statement_excel_node_list)}")
    except ValidationError as e:
        logging.error(f"Validation error for row {index}: {e}")

# Then, use the create_or_merge_neontology_node function to add these nodes to your Neo4j database
for node_data in learning_statement_excel_node_list:
    try:
        logging.pedantic(f"Creating or merging node: {node_data}")
        neon.create_or_merge_neontology_node(node_data['node'], operation='create')
    except Exception as e:
        logging.error(f"Error in processing node: {e}")

In [ ]:
# Create relationships between TopicNode and TopicLessonNode
relationship_list = []
for topic_lesson_node in topic_lesson_excel_node_list:
    logging.debug(f"Processing topic lesson node: {topic_lesson_node}")
    topic_lesson_node_id = topic_lesson_node['node'].topic_lesson_id
    logging.pedantic(f"Processing topic lesson node ID: {topic_lesson_node_id}")
    lesson_nodes = neo4j.find_nodes_by_label_and_properties(neo4j_driver, 'Lesson', {'topic_lesson_id': topic_lesson_node_id})
    logging.pedantic(f"Lesson nodes found: {lesson_nodes}")
    if not lesson_nodes:  # Check if the list is empty
        logging.error(f"No lesson node found for ID {topic_lesson_node_id}")

    topic_node_id = topic_lesson_node['helper']['topic_lesson_topic_id']
    logging.pedantic(f"Processing topic node ID: {topic_node_id}")
    topic_nodes = neo4j.find_nodes_by_label_and_properties(neo4j_driver, 'Topic', {'topic_id': topic_node_id})
    logging.pedantic(f"Topic nodes found: {topic_nodes}")
    if not topic_nodes:  # Check if the list is empty
        logging.error(f"No topic node found for ID {topic_node_id}")
    
    # Assuming only one node is expected for each query
    lesson_node_properties = dict(lesson_nodes[0])  # Extract properties from the Neo4j Node
    logging.pedantic(f"Lesson node properties: {lesson_node_properties}")
    topic_node_properties = dict(topic_nodes[0])    # Extract properties from the Neo4j Node
    logging.pedantic(f"Topic node properties: {topic_node_properties}")

    # Create instances of TopicNode and TopicLessonNode using the extracted properties
    logging.pedantic(f"Creating instances of TopicNode and TopicLessonNode")
    neontology_lesson_node = TopicLessonNode(**lesson_node_properties)
    logging.pedantic(f"Neontology lesson node: {neontology_lesson_node}")
    neontology_topic_node = TopicNode(**topic_node_properties)
    logging.pedantic(f"Neontology topic node: {neontology_topic_node}")

    # Create the relationship
    logging.pedantic(f"Creating relationship between {neontology_topic_node} and {neontology_lesson_node}")
    topic_has_lesson_relationship = TopicIncludesTopicLesson(source=neontology_topic_node, target=neontology_lesson_node)
    logging.pedantic(f"Relationship created: {topic_has_lesson_relationship}")

    # Merge or create relationship in the database
    logging.pedantic(f"Merging or creating relationship: {topic_has_lesson_relationship}")
    neon.create_or_merge_neontology_relationship(topic_has_lesson_relationship, operation='merge')
    logging.pedantic(f"Relationship merged or created: {topic_has_lesson_relationship}")

In [ ]:
logging = logger.get_logger(name='logger_tool', log_level='DEBUG', log_file='new.log')

# Create relationships between TopicNode and LearningStatementNode and LessonNode and LearningStatementNode
# Assuming only one node is expected for each query
for learning_statement_node in learning_statement_excel_node_list:
    logging.info(f"Processing learning statement node: {learning_statement_node}")
    learning_statement_node_id = learning_statement_node['node'].lesson_learning_statement_id

    logging.pedantic(f"Processing learning statement node ID: {learning_statement_node_id}")
    learning_statement_nodes = neo4j.find_nodes_by_label_and_properties(neo4j_driver, 'LearningStatement', {'lesson_learning_statement_id': learning_statement_node_id})
    if not learning_statement_nodes:  # Check if the list is empty
        logging.error(f"No learning statement node found for ID {learning_statement_node_id}")
        continue
    logging.pedantic(f"Learning statement nodes found: {learning_statement_nodes}")
    learning_statement_node_properties = dict(learning_statement_nodes[0])  # Extract properties from the Neo4j Node
    logging.pedantic(f"Learning statement node properties: {learning_statement_node_properties}")
    
    topic_node_id = learning_statement_node['helper']['lesson_learning_statement_topic_id']
    logging.pedantic(f"Processing topic node ID: {topic_node_id}")
    topic_nodes = neo4j.find_nodes_by_label_and_properties(neo4j_driver, 'Topic', {'topic_id': topic_node_id})
    if not topic_nodes:  # Check if the list is empty
        logging.error(f"No topic node found for ID {topic_node_id}")
        continue
    logging.pedantic(f"Topic nodes found: {topic_nodes}")
    topic_node_properties = dict(topic_nodes[0])    # Extract properties from the Neo4j Node
    logging.pedantic(f"Topic node properties: {topic_node_properties}")    

    topic_lesson_node_id = learning_statement_node['helper']['lesson_learning_statement_lesson_id']
    logging.pedantic(f"Processing topic lesson node ID: {topic_lesson_node_id}")
    topic_lesson_nodes = neo4j.find_nodes_by_label_and_properties(neo4j_driver, 'Lesson', {'topic_lesson_id': topic_lesson_node_id})
    if not topic_lesson_nodes:
        logging.error(f"No topic lesson node found for ID {topic_lesson_node_id}")
        continue
    logging.pedantic(f"Topic lesson nodes found: {topic_lesson_nodes}")
    lesson_node_properties = dict(topic_lesson_nodes[0])  # Extract properties from the Neo4j Node
    logging.pedantic(f"Lesson node properties: {lesson_node_properties}")
    
    # Create instances of TopicNode and TopicLessonNode using the extracted properties
    logging.pedantic(f"Creating instances of TopicNode and TopicLessonNode and LearningStatementNode")
    neontology_lesson_node = TopicLessonNode(**lesson_node_properties)
    logging.pedantic(f"Neontology lesson node: {neontology_lesson_node}")
    neontology_topic_node = TopicNode(**topic_node_properties)
    logging.pedantic(f"Neontology topic node: {neontology_topic_node}")
    neontology_learning_statement_node = LearningStatementNode(**learning_statement_node_properties)
    logging.pedantic(f"Neontology learning statement node: {neontology_learning_statement_node}")

    # Create the relationship between learning statement and lesson
    logging.pedantic(f"Creating relationship between {neontology_learning_statement_node} and {neontology_lesson_node}")
    statement_in_lesson_relationship = LearningStatementPartOfTopicLesson(source=neontology_learning_statement_node, target=neontology_lesson_node)
    logging.pedantic(f"Relationship created: {statement_in_lesson_relationship}")

    # Create the relationship between learning statement and topic
    logging.pedantic(f"Creating relationship between {neontology_learning_statement_node} and {neontology_topic_node}")
    topic_has_learning_statement_relationship = TopicIncludesLearningStatement(source=neontology_topic_node, target=neontology_learning_statement_node)
    logging.pedantic(f"Relationship created: {topic_has_learning_statement_relationship}")

    # Merge or create relationships in the database
    logging.pedantic(f"Merging or creating relationship: {statement_in_lesson_relationship}")
    neon.create_or_merge_neontology_relationship(statement_in_lesson_relationship, operation='merge')
    logging.pedantic(f"Relationship merged or created: {statement_in_lesson_relationship}")
    logging.pedantic(f"Merging or creating relationship: {topic_has_learning_statement_relationship}")
    neon.create_or_merge_neontology_relationship(topic_has_learning_statement_relationship, operation='merge')
    logging.pedantic(f"Relationship merged or created: {topic_has_learning_statement_relationship}")
    logging.pedantic(f"Relationships created between {neontology_learning_statement_node} and {neontology_lesson_node} and {neontology_topic_node}")

In [ ]:
# Create SchoolNodes from a list
schools = [
    SchoolNode(school_id='FPGS', school_name='Fort Pitt Grammar School', school_org_type='Academy', school_address='Fort Pitt Grammar School, Fort Pitt Hill, Chatham, Kent, ME4 6TJ', school_website='https://www.fortpitt.medway.sch.uk/'),
    SchoolNode(school_id='KCAR', school_name='King Charles School', school_org_type='Academy', school_address='King Charles School, Holly Hall Road, Dudley, West Midlands, DY2 0TZ', school_website='https://www.kingcharlesschool.co.uk/'),
    SchoolNode(school_id='KEV', school_name='Kevlar Academy', school_org_type='Academy', school_address='Kevlar Academy, 123 Fake Street, London, SW1A 1AA', school_website='https://www.kevlaracademy.com/')
]
# Now create a routine to create schools from a list
for school in schools:
    neon.create_or_merge_neontology_node(school, operation='create')

# Create DepartmentNodes from a list
departments = [
    DepartmentNode(department_id='SCI', department_name='Science'),
    DepartmentNode(department_id='MAT', department_name='Mathematics'),
    DepartmentNode(department_id='ENG', department_name='English'),
    DepartmentNode(department_id='HIS', department_name='History')
]
# Now create a routine to create departments from a list
for department in departments:
    neon.create_or_merge_neontology_node(department, operation='create')

# Create KeyStageNodes from a list
key_stages = [
    KeyStageNode(key_stage_level=3, key_stage='KS3'),
    KeyStageNode(key_stage_level=4, key_stage='KS4'),
    KeyStageNode(key_stage_level=5, key_stage='KS5')
]
# Now create a routine to create key stages from a list
for key_stage in key_stages:
    neon.create_or_merge_neontology_node(key_stage, operation='create')
# Create SubjectNodes from a list
subjects = [
    SubjectNode(subject_id='SCI', subject_name='Science'),
    SubjectNode(subject_id='PHY', subject_name='Physics'),
    SubjectNode(subject_id='CHE', subject_name='Chemistry'),
    SubjectNode(subject_id='BIO', subject_name='Biology'),
    SubjectNode(subject_id='MAT', subject_name='Mathematics'),
    SubjectNode(subject_id='ENG', subject_name='English'),
    SubjectNode(subject_id='HIS', subject_name='History')
]
# Now create a routine to create subjects from a list
for subject in subjects:
    neon.create_or_merge_neontology_node(subject, operation='create')


# Create relationships between ALL SchoolNodes and DepartmentNodes
for school_node in schools:
    logging.debug(f"Processing school node: {school_node}")
    school_node_id = school_node.school_id
    logging.pedantic(f"Processing school node ID: {school_node_id}")
    school_nodes = neo4j.find_nodes_by_label_and_properties(neo4j_driver, 'School', {'school_id': school_node_id})
    logging.pedantic(f"School nodes found: {school_nodes}")
    if not school_nodes:  # Check if the list is empty
        logging.error(f"No school node found for ID {school_node_id}")
        continue
    school_node_properties = dict(school_nodes[0])  # Extract properties from the Neo4j Node
    logging.pedantic(f"School node properties: {school_node_properties}")

    for department_node in departments:
        logging.debug(f"Processing department node: {department_node}")
        department_node_id = department_node.department_id
        logging.pedantic(f"Processing department node ID: {department_node_id}")
        department_nodes = neo4j.find_nodes_by_label_and_properties(neo4j_driver, 'Department', {'department_id': department_node_id})
        logging.pedantic(f"Department nodes found: {department_nodes}")
        if not department_nodes:  # Check if the list is empty
            logging.error(f"No department node found for ID {department_node_id}")
            continue
        department_node_properties = dict(department_nodes[0])  # Extract properties from the Neo4j Node
        logging.pedantic(f"Department node properties: {department_node_properties}")

        # Create instances of SchoolNode and DepartmentNode using the extracted properties
        logging.pedantic(f"Creating instances of SchoolNode and DepartmentNode")
        neontology_school_node = SchoolNode(**school_node_properties)
        logging.pedantic(f"Neontology school node: {neontology_school_node}")
        neontology_department_node = DepartmentNode(**department_node_properties)
        logging.pedantic(f"Neontology department node: {neontology_department_node}")

        # Create the relationship
        logging.pedantic(f"Creating relationship between {neontology_school_node} and {neontology_department_node}")
        school_has_department_relationship = SchoolHasDepartment(source=neontology_school_node, target=neontology_department_node)
        logging.pedantic(f"Relationship created: {school_has_department_relationship}")

        # Merge or create relationship in the database
        logging.pedantic(f"Merging or creating relationship: {school_has_department_relationship}")
        neon.create_or_merge_neontology_relationship(school_has_department_relationship, operation='merge')
        logging.pedantic(f"Relationship merged or created: {school_has_department_relationship}")

In [ ]:
# Create relationships between DepartmentNodes and SubjectNodes
relationship_list = []
for department_node in departments:
    logging.debug(f"Processing department node: {department_node}")
    department_node_id = department_node.department_id
    logging.pedantic(f"Processing department node ID: {department_node_id}")
    department_nodes = neo4j.find_nodes_by_label_and_properties(neo4j_driver, 'Department', {'department_id': department_node_id})
    logging.pedantic(f"Department nodes found: {department_nodes}")
    if not department_nodes:  # Check if the list is empty
        logging.error(f"No department node found for ID {department_node_id}")
        continue
    department_node_properties = dict(department_nodes[0])  # Extract properties from the Neo4j Node
    logging.pedantic(f"Department node properties: {department_node_properties}")

    for subject_node in subjects:
        logging.debug(f"Processing subject node: {subject_node}")
        subject_node_id = subject_node.subject_id
        logging.pedantic(f"Processing subject node ID: {subject_node_id}")
        subject_nodes = neo4j.find_nodes_by_label_and_properties(neo4j_driver, 'Subject', {'subject_id': subject_node_id})
        logging.pedantic(f"Subject nodes found: {subject_nodes}")
        if not subject_nodes:  # Check if the list is empty
            logging.error(f"No subject node found for ID {subject_node_id}")
            continue
        subject_node_properties = dict(subject_nodes[0])  # Extract properties from the Neo4j Node
        logging.pedantic(f"Subject node properties: {subject_node_properties}")

        # Create instances of DepartmentNode and SubjectNode using the extracted properties
        logging.pedantic(f"Creating instances of DepartmentNode and SubjectNode")
        neontology_department_node = DepartmentNode(**department_node_properties)
        logging.pedantic(f"Neontology department node: {neontology_department_node}")
        neontology_subject_node = SubjectNode(**subject_node_properties)
        logging.pedantic(f"Neontology subject node: {neontology_subject_node}")

        # Create the relationship
        logging.pedantic(f"Creating relationship between {neontology_department_node} and {neontology_subject_node}")
        school_offers_subject_relationship = DepartmentProvidesSubject(source=neontology_department_node, target=neontology_subject_node)
        logging.pedantic(f"Relationship created: {school_offers_subject_relationship}")

        # Merge or create relationship in the database
        logging.pedantic(f"Merging or creating relationship: {school_offers_subject_relationship}")
        neon.create_or_merge_neontology_relationship(school_offers_subject_relationship, operation='merge')
        logging.pedantic(f"Relationship merged or created: {school_offers_subject_relationship}")

In [ ]:
# Create relationships between SubjectNodes, KeyStageNodes and TopicNodes using the excel helper data
relationship_list = []
for topic_node in topic_excel_node_list:
    logging.debug(f"Processing topic node: {topic_node}")
    topic_node_id = topic_node['node'].topic_id
    logging.pedantic(f"Processing topic node ID: {topic_node_id}")
    topic_nodes = neo4j.find_nodes_by_label_and_properties(neo4j_driver, 'Topic', {'topic_id': topic_node_id})
    logging.pedantic(f"Topic nodes found: {topic_nodes}")
    if not topic_nodes:  # Check if the list is empty
        logging.error(f"No topic node found for ID {topic_node_id}")
        continue

    topic_node_properties = dict(topic_nodes[0])  # Extract properties from the Neo4j Node
    logging.pedantic(f"Topic node properties: {topic_node_properties}")
    
    # Get the subject ID and Key Stage from the excel helper data
    subject_node_id = topic_node['helper']['topic_subject']
    key_stage_level = topic_node['helper']['topic_key_stage']

    logging.pedantic(f"Processing subject node ID: {subject_node_id}")
    subject_nodes = neo4j.find_nodes_by_label_and_properties(neo4j_driver, 'Subject', {'subject_id': subject_node_id})
    logging.pedantic(f"Subject nodes found: {subject_nodes}")
    if not subject_nodes:  # Check if the list is empty
        logging.error(f"No subject node found for ID {subject_node_id}")
        continue

    subject_node_properties = dict(subject_nodes[0])  # Extract properties from the Neo4j Node
    logging.pedantic(f"Subject node properties: {subject_node_properties}")

    logging.pedantic(f"Processing key stage level: {key_stage_level}")
    key_stage_nodes = neo4j.find_nodes_by_label_and_properties(neo4j_driver, 'KeyStage', {'key_stage_level': key_stage_level})
    logging.pedantic(f"Key stage nodes found: {key_stage_nodes}")
    if not key_stage_nodes:  # Check if the list is empty
        logging.error(f"No key stage node found for level {key_stage_level}")
        continue

    key_stage_node_properties = dict(key_stage_nodes[0])  # Extract properties from the Neo4j Node
    logging.pedantic(f"Key stage node properties: {key_stage_node_properties}")

    # Create relationship between the subject and key stage
    logging.pedantic(f"Creating relationship between {subject_node_properties} and {key_stage_node_properties}")
    subject_for_key_stage_relationship = SubjectForKeyStage(source=subject_node_properties, target=key_stage_node_properties)
    logging.pedantic(f"Relationship created: {subject_for_key_stage_relationship}")

    # Create relationship between the topic and subject
    logging.pedantic(f"Creating relationship between {topic_node_properties} and {subject_node_properties}")
    topic_for_subject_relationship = SubjectIncludesTopic(source=subject_node_properties, target=topic_node_properties)
    logging.pedantic(f"Relationship created: {topic_for_subject_relationship}")

    # Create relationship between the topic and key stage
    logging.pedantic(f"Creating relationship between {topic_node_properties} and {key_stage_node_properties}")
    topic_for_key_stage_relationship = KeyStageIncludesTopic(source=key_stage_node_properties, target=topic_node_properties)
    logging.pedantic(f"Relationship created: {topic_for_key_stage_relationship}")

    # Merge or create relationships in the database
    logging.pedantic(f"Merging or creating relationship: {subject_for_key_stage_relationship}")
    neon.create_or_merge_neontology_relationship(subject_for_key_stage_relationship, operation='merge')
    logging.pedantic(f"Relationship merged or created: {subject_for_key_stage_relationship}")

    logging.pedantic(f"Merging or creating relationship: {topic_for_subject_relationship}")
    neon.create_or_merge_neontology_relationship(topic_for_subject_relationship, operation='merge')
    logging.pedantic(f"Relationship merged or created: {topic_for_subject_relationship}")
    
    logging.pedantic(f"Merging or creating relationship: {topic_for_key_stage_relationship}")
    neon.create_or_merge_neontology_relationship(topic_for_key_stage_relationship, operation='merge')
    logging.pedantic(f"Relationship merged or created: {topic_for_key_stage_relationship}")
    
    logging.pedantic(f"Relationships created between {topic_node_properties} and {subject_node_properties} and {key_stage_node_properties}")


In [ ]:
# Create